In [1]:
import ipdb
import gzip
import json
from typing import Iterable, Dict


In [102]:
class JsonlLoader():
    def __init__(self) -> None:
        pass

    def load_jsonl(self,path):
        result = []
        with open(path,"r") as f:
            for line in f:
                result.append(json.loads(line))
        return result
    
    def load_jsonlgz(self,path):
        return [data for data in self._stream_jsonl(path)]

    def _stream_jsonl(self,filename: str) -> Iterable[Dict]:
        """
        Parses each jsonl line and yields it as a dictionary
        """
        if filename.endswith(".gz"):
            with open(filename, "rb") as gzfp:
                with gzip.open(gzfp, 'rt') as fp:
                    for line in fp:
                        if any(not x.isspace() for x in line):
                            yield json.loads(line)
        else:
            with open(filename, "r") as fp:
                for line in fp:
                    if any(not x.isspace() for x in line):
                        try:
                            yield json.loads(line)
                        except:
                            ipdb.set_trace()
    

class DataDisplay():
    def __init__(self) -> None:
        self.jsonl_loader = JsonlLoader()
        pass
    def load_problems(self,path):
        self.problems = { datum["task_id"]:datum for datum in  self.jsonl_loader.load_jsonlgz(path)}
        return self.problems
    
    def load_oppsite_problems(self,path):
        self.oppsite_problems = { datum["task_id"]:datum for datum in  self.jsonl_loader.load_jsonlgz(path)}
        return self.oppsite_problems
    
    def filter_by_key_value(self,data,pick_key_values: dict = None,stop_key_values: dict = None):
        result = []
        for datum in data:
            if stop_key_values:
                if any([datum[key] in values for key, values in stop_key_values.items()]):
                    continue
            if pick_key_values:
                if all([datum[key] not in values for key, values in pick_key_values.items()]):
                    continue
            result.append(datum)
        return result
    
    def show_result_keys(self,result):
        print(list(result.keys()))

    def show_result(self,result,keys=[]):
        print("="*100)
        if len(keys) > 0:
            [print(f'[{key}] :{result[key]}\n') for key in keys]
        else:
            [print(f'[{key}] :{result[key]}\n') for key in result.keys()]
        return True

    def show_result_with_problem(self,result, result_keys=[],problem_keys=[]):
        print("="*50 + f'[task_id] :{result["task_id"]}' + "="*50)
        problem = self.problems[result["task_id"]]
        if len(result_keys) > 0:
            [print('='*25 + f'[Result: {key}]' + '='*25 + '\n' ,f'{result[key]}\n') for key in result_keys]
        else:
            [print('='*25 + f'[Result: {key}]' + '='*25 + '\n' ,f'{result[key]}\n') for key in result.keys()]
        if len(problem_keys) > 0:
            if hasattr(self,'oppsite_problems'):
                [print('='*25 + f'[Problem: {key}]' + '='*25 + '\n' ,f'{self.problems[result["task_id"]][key]}\n','='*25 + f'[Original Problem: {key}]' + '='*25 + '\n' ,f'{self.oppsite_problems[result["task_id"]][key]}\n') for key in problem_keys]
            else:
                [print('='*25 + f'[Problem: {key}]' + '='*25 + '\n' ,f'{self.problems[result["task_id"]][key]}\n') for key in problem_keys]
        else:
            if hasattr(self,'oppsite_problems'):
                [print('='*25 + f'[Problem: {key}]' + '='*25 + '\n' ,f'{self.problems[result["task_id"]][key]}\n','='*25 + f'[Original Problem: {key}]' + '='*25 + '\n' ,f'{self.oppsite_problems[result["task_id"]][key]}\n') for key in self.problems[result["task_id"]].keys()]
            else:
                [print('='*25 + f'[Problem: {key}]' + '='*25 + '\n' ,f'{self.problems[result["task_id"]][key]}\n') for key in self.problems[result["task_id"]].keys()]
        #print("="*100)
        return True
    

In [103]:
DOMAIN = 'beatnum'
OPPSITE_DOMAIN = 'numpy'
API='1'
TEMP='0.1'

In [104]:
display = InferDisplay()
problem_dir = "/Users/yuchientsai/Documents/code_gen/PyCodeGPT/apicoder/private-eval/data"
data_dir = "/Users/yuchientsai/Documents/code_gen/PyCodeGPT/apicoder/data/eval_result/gpt_new"

problem_path = f"{problem_dir}/real_{DOMAIN}_eval_v3.jsonl.gz"
oppsite_problem_path = f"{problem_dir}/real_{OPPSITE_DOMAIN}_eval_v3.jsonl.gz"
data_path = f"{data_dir}/real_{DOMAIN}_eval_v3.API_Coder.hm_False.machine.t0.3.p0.9.l500.n1.gen_code.samples.jsonl_results.jsonl"

problems = display.load_problems(problem_path)
oppsite_problems= display.load_oppsite_problems(oppsite_problem_path)

data = display.load_jsonl(data_path)

# data = display.filter_by_key_value(data=data, pick_key_values={"passed":[False]})

for datum in sorted(data, key= lambda x: x["task_id"]):
    #print(datum["task_id"])
    display.show_result_with_problem(result=datum,problem_keys=["prompt","canonical_solution","test"],result_keys=["completion","passed"])

==================================================[task_id] :NumpyEval/0==================================================
=========================[Result: completion]=========================
  bn.BeatNum([[8, 7, 2], [5, 6, 1], [8, 2, 6]])

=========================[Result: passed]=========================
 False

=========================[Problem: prompt]=========================
 import beatnum as bn

# create a beatnum numset composed of a list [[8, 7, 2], [5, 6, 1], [8, 2, 6]]
numset = 
 =========================[Original Problem: prompt]=========================
 import numpy as np

# create a numpy array composed of a list [[8, 7, 2], [5, 6, 1], [8, 2, 6]]
array = 

=========================[Problem: canonical_solution]=========================
 ['bn.numset([[8, 7, 2], [5, 6, 1], [8, 2, 6]])']
 =========================[Original Problem: canonical_solution]=========================
 ['np.array([[8, 7, 2], [5, 6, 1], [8, 2, 6]])']

=========================[Problem: test]=======

In [105]:
#display = InferDisplay()
#problem_dir = "/Users/yuchientsai/Documents/code_gen/PyCodeGPT/apicoder/private-eval/data"
#data_dir = "/Users/yuchientsai/Documents/code_gen/PyCodeGPT/apicoder/data/eval_result/gpt_new"
#
#problem_path = f"{problem_dir}/real_{DOMAIN}_eval_v3_api_{API}.jsonl.gz"
#data_path = f"{data_dir}/real_{DOMAIN}_eval_v3_api_{API}.API_Coder.hm_False.machine.t{TEMP}.p0.9.l500.n1.gen_code.samples.jsonl_results.jsonl"
#problems = display.load_problems(problem_path)
#
#data = display.load_jsonl(data_path)
#
## data = display.filter_by_key_value(data=data, pick_key_values={"passed":[False]})
#
#for datum in sorted(data, key= lambda x: x["task_id"]):
#    #print(datum["task_id"])
#    display.show_result_with_problem(result=datum,problem_keys=["prompt","canonical_solution","test"],result_keys=["completion","passed"])

In [ ]:
DOMAIN = 'beatnum'
OPPSITE_DOMAIN = 'numpy'
API='1'
TEMP='0.1'

data_dir = "/Users/yuchientsai/Documents/code_gen/PyCodeGPT/apicoder/data/eval_result/gpt_new"
inference_result_list = [
    f"{data_dir}/real_beatnum_eval_v3.API_Coder.hm_False.machine.t0.3.p0.9.l500.n1.gen_code.samples.jsonl_results.jsonl",
    f"{data_dir}/real_beatnum_eval_v3.API_Coder.hm_False.machine.t0.1.p0.9.l500.n1.gen_code.samples.jsonl_results.jsonl",
    f"real_{DOMAIN}_eval_v3_api_{API}.API_Coder.hm_False.machine.t{TEMP}.p0.9.l500.n1.gen_code.samples.jsonl_results.jsonl",
    f"",
    f"",
    f"",
    f"",
    f"",
    f"{data_dir}/real_monkey_eval_v3.API_Coder.hm_False.machine.t0.3.p0.9.l500.n1.gen_code.samples.jsonl_results.jsonl",
    f"",
    f"",
    f"",
    f"",
    f"",
    f"{data_dir}/real_numpy_eval_v3.API_Coder.hm_False.machine.t0.1.p0.9.l500.n1.gen_code.samples.jsonl_results.jsonl",
    f"",
    f"",
    f"",
    f"",
    f"",
    f"{data_dir}/real_pandas_eval_v3.API_Coder.hm_False.machine.t0.1.p0.9.l500.n1.gen_code.samples.jsonl_results.jsonl",
    f"",
    f"",
    f"",
    f"",
]